# KBO
### Selenium을 사용하여 데이터 수집하는 방법을 살펴보겠습니다.


In [1]:
!dir

 C 드라이브의 볼륨: OS
 볼륨 일련 번호: 921A-68D9

 C:\20220103_lab\14.python 디렉터리

2022-04-08  오후 03:20    <DIR>          .
2022-04-08  오후 03:20    <DIR>          ..
2022-04-08  오후 03:20    <DIR>          .ipynb_checkpoints
2022-04-05  오후 03:23    <DIR>          apple
2022-04-05  오후 03:16             1,187 bigbang.txt
2022-04-06  오후 07:01    <DIR>          data
2022-04-04  오후 07:07            78,667 encore_220404.ipynb
2022-04-06  오후 07:06         5,322,810 encore_220405.ipynb
2022-04-05  오후 05:47             2,863 encore_220405_assignment_melon.ipynb
2022-04-06  오후 07:51            24,470 encore_220406.ipynb
2022-04-07  오후 06:13           945,662 encore_220407.ipynb
2022-04-08  오후 03:20           338,751 encore_220408.ipynb
2022-04-07  오전 09:38             6,648 kbo.pkl
2022-04-05  오후 05:36    <DIR>          melon
2022-04-06  오후 02:24                 0 movetext.TXT
2022-04-08  오후 03:20             8,356 Selenium을 사용한 KBO 데이터 수집.ipynb
2022-04-06  오전 10:33             1,244 star.py
2022-04-06  오후 1

In [4]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

* 정규식 패턴을 사용합니다. 
    * 아래 정규식 패턴은 playerId의 숫자값을 추출하는 정규식입니다.

In [9]:
pattern = re.compile("playerId=([0-9]+)")

* Selenium을 사용하기 위해서는 브라우저의 드라이버를 설정해야 합니다. 
    * 이번 예제는 크롬 브라우저를 사용하기 때문에 크롬 드라이버를 선택해줍니다. 
    * 다운로드
    * https://chromedriver.chromium.org/downloads

In [10]:
s = Service("c:/chrome/chromedriver.exe")

driver = webdriver.Chrome(service=s)

In [14]:
driver.get("http://www.koreabaseball.com")

By.CSS_SELECTOR   ->   #popupVideo > img

In [15]:
driver.find_element(by=By.CSS_SELECTOR, 
                    value="#popupVideo > img").click()

In [6]:
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(8)
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(10)

By.CSS_SELECTOR   ->   #lnb > li:nth-child(4) > a

In [16]:
driver.find_element(by=By.CSS_SELECTOR, 
                     value="#lnb > li:nth-child(4) > a").click()

In [17]:
select_team = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"

In [18]:
select_page = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"

In [20]:
import time

In [21]:
playid = []
for x in range(2,12):
    for_1 = select_team.format(x)
    driver.find_element(by=By.CSS_SELECTOR, value=for_1).click()
    time.sleep(2)
    #playid.extend(pattern.findall(driver.page_source))
    for y in range(1,6):
        f2 = select_page.format(y)
        try:
            driver.find_element(by=By.CSS_SELECTOR, value=f2).click()
            time.sleep(1)
            playid.extend(pattern.findall(driver.page_source))
        except Exception as e:
            print (e)
        time.sleep(2)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#cphContents_cphContents_cphContents_ucPager_btnNo5"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C07413+2389011]
	Ordinal0 [0x00B99F61+1941345]
	Ordinal0 [0x00A8C658+837208]
	Ordinal0 [0x00AB91DD+1020381]
	Ordinal0 [0x00AB949B+1021083]
	Ordinal0 [0x00AE6032+1204274]
	Ordinal0 [0x00AD4194+1130900]
	Ordinal0 [0x00AE4302+1196802]
	Ordinal0 [0x00AD3F66+1130342]
	Ordinal0 [0x00AAE546+976198]
	Ordinal0 [0x00AAF456+980054]
	GetHandleVerifier [0x00DB9632+1727522]
	GetHandleVerifier [0x00E6BA4D+2457661]
	GetHandleVerifier [0x00C9EB81+569713]
	GetHandleVerifier [0x00C9DD76+566118]
	Ordinal0 [0x00BA0B2B+1968939]
	Ordinal0 [0x00BA5988+1989000]
	Ordinal0 [0x00BA5A75+1989237]
	Ordinal0 [0x00BAECB1+2026673]
	BaseThreadInitThunk [0x75F2FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77387A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77387A4E+238]

Message: no such element: Unable 

In [ ]:
len(playid)

In [ ]:
import pickle

In [ ]:
with open("./kbo.pkl", "wb") as f:
    pickle.dump(playid,f)

### 선수들의 세부 항목들을 수집합니다.

In [22]:
r = requests.get("https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=62356")

* 이미지를 다운로드 받고 싶다면 아래 코드를 사용합니다.

In [23]:
request.urlretrieve("http:"+BS(r.text).find("div",\
            class_ = "player_basic").find("img")['src'], "./김규민.jpg")

('./김규민.jpg', <http.client.HTTPMessage at 0x1bb5ebab2b0>)

### 사용자 함수를 작성하여 선수들의 세부 정보를 수집합니다.

In [26]:
def playerID(id_):
    play_dict = {}
    url = "https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId={}".format(id_)
    r = requests.get(url)
    #print (r.url)
    for x in BS(r.text).find("div",\
                class_ = "player_basic").findAll("li"):
        rt = [y.strip() for y in x.text.strip().split(":")]
        play_dict[rt[0]] = rt[1]
    play_dict['Team'] = BS(r.text).find("h4", id="h4Team").text
    return play_dict

In [27]:
playid[0]

'69158'

In [29]:
playerID(playid[1])

{'선수명': '강효종',
 '등번호': 'No.43',
 '생년월일': '2002년 10월 14일',
 '포지션': '투수(우투우타)',
 '신장/체중': '184cm/86kg',
 '경력': '저동초(일산서구리틀)-충암중-충암고',
 '입단 계약금': '20000만원',
 '연봉': '3000만원',
 '지명순위': '21 LG 1차',
 '입단년도': '21LG',
 'Team': 'LG 트윈스'}